# PART 2

# Creating table along with latitude and the longitude coordinates of each neighborhood

In [3]:
import urllib.request
import pandas as pd
import numpy as np

#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Query the website and return the html to the variable 'page'
page=urllib.request.urlopen(wiki)

#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page)

#Selecting table with class='wikitable sortable'
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['Postalcode', 'Borough', 'Neighborhood'])
df.head()

#Dropping rows where BOUROUGH = Not assigned
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)

#Assigning value of Neighborhood same as Borough if Neighborhood == "Not assigned"
#df['Neighborhood'] = np.where(df['Neighborhood'] =="Not assigned", df['Borough'], df['Neighborhood'])

#Creating new dataframe grouped by Postalcode & Borough
df_new =df.groupby(['Postalcode','Borough'],as_index=False).agg(lambda x: ','.join(x))

## Loading data from CSV file & merging data

In [8]:
#Loading geographical coordinates of each postal code from csv file
df_geo= pd.read_csv('https://cocl.us/Geospatial_data')

#Renaming column for merging purpose
df_geo.rename(columns = {'Postal Code': 'Postalcode'}, inplace=True)

#Merging two dataframes on POSTALCODE
df_merged=pd.merge(df_new, df_geo, on='Postalcode')
df_merged.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
